### Import necassary libraries

In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt

torch.__version__

### Setup device 

In [ ]:
device='cuda' if torch.cuda.is_available() else 'cpu'

### Create Data

In [ ]:
#create the random data sample for a line
weight  = 0.3
bias    = 0.9

start = 0
end   = 1
step  = 0.01  

X = torch.arange(start,end,step).unsqueeze(dim=1)
y = weight*X+bias

#splitting the data into train and test

train_split = int(0.8*len(X))
X_train = X[:train_split]
y_train = y[:train_split]

X_test  = X[train_split:]
y_test  = y[train_split:]


### Visualize the data

In [ ]:
def plot_predictions(xtrain, ytrain, xtest, ytest, predictions=None):
    plt.figure(figsize=(10,10))
    plt.scatter(xtrain, ytrain, c='b', s=4, label='Training data')
    plt.scatter(xtest, ytest, c='r', s=4, label='Testing data')
    
    if predictions is not None:
        plt.scatter(xtest, predictions, c='y', s=4, label='Predictions')
        
    plt.legend()
    plt.show()


In [ ]:

plot_predictions(X_train,y_train,X_test,y_test)

### Building the Model

In [ ]:
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
    #manually setting weights and bias
        self.weights = nn.Parameter(torch.randn(1, dtype=torch.float,requires_grad=True))
        self.bias    = nn.Parameter(torch.randn(1, dtype=torch.float,requires_grad=True))

    def forward(self,x:torch.Tensor)-> torch.Tensor:
        return self.weights*x + self.bias
    
    #next approach without manually creating the weights and bias
#         self.linear_layer = nn.Linear(in_feature=1, out_feature=1)
    
#     def forward(self, x:torch.Tensor)-> torch.Tensor:
#         return self.linear_layer(x)

In [ ]:
model=LinearRegressionModel()
model.state_dict()

In [ ]:
model.to(device)
next(model.parameters()).device

### Hyperparameters

In [ ]:
learning_rate = 0.01
epochs =300

### Model Training

In [ ]:
loss_fn=nn.L1Loss()
optimizer=torch.optim.SGD(params=model.parameters(),lr=learning_rate)

In [ ]:
X_train = X_train.to(device)
X_test = X_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)


for epoch in range(epochs):
    
    model.train()
    
    #forward pass
    y_pred=model(X_train)
    
    #calcualte loss
    training_loss=loss_fn(y_pred,y_train)
    
    #zero grad
    optimizer.zero_grad()
    
    #backward propagation
    training_loss.backward()
    
    #update weights
    optimizer.step()
    
    #testing loop
    model.eval()
    with torch.inference_mode():
        test_pred=model(X_test)
        test_loss=loss_fn(test_pred,y_test)
        
    
    if epoch % 15 ==0:
        print(f"{epoch}: training loss={training_loss} | test loss={test_loss}")

In [ ]:
list(model.parameters())

### making predictions

In [ ]:
with torch.inference_mode():
    pred=model(X_test)

In [ ]:
plot_predictions(X_train,y_train,X_test,y_test,pred)

### Saving the model

In [ ]:
from pathlib import Path

MODEL_PATH=Path("models")
MODEL_PATH.mkdir(parents=True, exist_ok=True)

MODEL_NAME='exercise.pth'
MODEL_SAVE_PATH=MODEL_PATH/MODEL_NAME

print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=model.state_dict(), # only saving the state_dict() only saves the models learned parameters
           f=MODEL_SAVE_PATH)